In [45]:
import pandas as pd
import re
import numpy as np
import datetime

## Step #1
### Import data from csv

In [46]:
data = pd.read_csv('Billboard Data.csv')

## Step #2 Tidy the data

### First use the melt function gather data by the week column so each week is a new row instead of a new column.

In [47]:
data=(data.melt(id_vars=["year","artist.inverted","track","genre","date.entered","date.peaked","time"],var_name=["week"],value_name="rank"))
data.head()

,year,artist.inverted,track,genre,date.entered,date.peaked,time,week,rank
0,2000,Destiny's Child,Independent Women Part I,Rock,9/23/2000,11/18/2000,3:38,x1st.week,78.0
1,2000,Santana,"Maria, Maria",Rock,2/12/2000,4/8/2000,4:18,x1st.week,15.0
2,2000,Savage Garden,I Knew I Loved You,Rock,10/23/1999,1/29/2000,4:07,x1st.week,71.0
3,2000,Madonna,Music,Rock,8/12/2000,9/16/2000,3:45,x1st.week,41.0
4,2000,"Aguilera, Christina",Come On Over Baby (All I Want Is You),Rock,8/5/2000,10/14/2000,3:38,x1st.week,57.0


### Then we convert the Week column to a number

In [48]:
data.update(pd.DataFrame({"week": np.ravel([list(map(int, re.findall(r'\d+', i))) for i in data["week"]])}))

### Then we create a new column for the variable week from the date entered and week column

In [ ]:
data['date.entered'] =  pd.to_datetime(data['date.entered'], format='%m/%d/%Y')
data=(data.assign(date= [data['date.entered'][i]+datetime.timedelta(weeks = data["week"][i]-1) for i in range(len(data["week"]))])
         .drop(['date.entered','date.peaked','week', "artist.inverted", "track"], axis=1)
         .sort_values('artist.inverted', ascending=True)
         .reset_index(drop=True))
data.head()

### Then we enter an ID column

In [ ]:
labels,unique=pd.factorize(list(data["track"]))
data["id"]=labels+1
data.reset_index(drop=True)
data.head()

### Then we split the data into two dataframes. One contains a record fo each song with a unique ID and the other contains that songs weekly ranking. 

In [ ]:
# creating a new dataframe for rank
df_rank=data.filter(["id", "date", "rank"]).dropna()
df_rank=data.sort_values(by=['id','date']).reset_index(drop=True)
# creating a new dataframe for song
df_song=data.filter(["id", "artist.inverted", "track","time"])
df_song=df_song.drop_duplicates('id').reset_index(drop=True)
df_rank.head(30)
 

In [ ]:
df_song.head(30)